In this notebook I am sharing my efficient way of making a dataset. Not sure if there are any bugs. Please enlighten me if you find some.

I have tried to make a dataset that can work with large batch sizes.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [ ]:
train = pickle.load(open('../input/ubiquant-market-prediction-half-precision-pickle/train.pkl', 'rb'))
# train = train.sample(frac=.001)
print(train.columns)
train.head()

In [ ]:
investment_ids = train.investment_id.unique()
investment_id_map = dict(zip(investment_ids, range(len(investment_ids))))
pickle.dump(investment_id_map, open('investment_id_map.pkl', 'wb'))

In [ ]:
def preprocess(df):
    return df

In [ ]:
train = preprocess(train)
train.head()

In [ ]:
features = [f'f_{i}' for i in range(300)]
target = 'target'

Train - Val should be split on time_id

In [ ]:
train_df = train
val_df = train

In [ ]:
train_df = train_df.reset_index()
val_df = val_df.reset_index()

# Dataset

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import catboost as cb
from sklearn.model_selection import train_test_split

import pdb

class UMPDataset(Dataset):
    def __init__(self,
                 idxs: pd.DataFrame,
                 ret_time_id:bool=False,
                 transforms=None):
        super().__init__()
        self.idxs = idxs
        self.ret_time_id = ret_time_id
        self.transforms = transforms
        
    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, idx):
        return idx
    

def process_df(df, idx, ret_time_id:bool=False, device:str='cpu'):
    x = df.loc[idx, features].values
    y = df.loc[idx, target].values.reshape(-1,1)
    x = torch.from_numpy(x).float().to(device)
    y = torch.from_numpy(y).float().to(device)
    sample = (x,y)
    if ret_time_id:
        return x, y, torch.from_numpy(df.loc[idx, 'time_id'].values.reshape(-1,1).astype(int)).long().to(device)
    return sample

In [ ]:
%%time
ds = UMPDataset(train_df, 
                transforms=None, 
                ret_time_id=False)

dl = DataLoader(
    dataset=ds,
    batch_size=4096*20,
    num_workers=2,
    shuffle=True
)

In [ ]:
%%time
idx = next(iter(dl))

In [ ]:
%%time
x, y, time_id = process_df(train_df, idx.cpu().numpy(), ret_time_id=True)
x.shape, y.shape, time_id.shape